Используя данные датасета, https://www.kaggle.com/datasets/asaniczka/tmdb-movies-dataset-2023-930k-movies/, реализовать задачу анализа аннотаций к фильмам. Результатом работы должен быть список слов из аннотаций и рецензий, влияющий на продвижение фильма.  

Шаги:

1. Подготовка данных

a.     Очистка текста от неалфавитных символов.

b.     Токенизация.

c.      Лемматизация.

d.     Удаление стоп-слов.

e.      Векторизация текста.

2.     Выбор признаков

3.     Обучение модели



Оформить выводы.

In [92]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from IPython.display import display
from sklearn.feature_selection import SelectKBest, f_regression
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [61]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [74]:
data = pd.read_csv("TMDB_movie_dataset_v11.csv")
data.head()

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,original_language,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,en,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili"
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,en,Interstellar,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,en,The Dark Knight,Batman raises the stakes in his war on crime. ...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin"
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish"
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,en,The Avengers,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian"


In [75]:
# Очистка текста от неалфавитных символов
import re
def clean_text(text):
    if isinstance(text, str):
      # Очистка текста от неалфавитных символов
      text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text

In [76]:
data['overview'] = data['overview'].apply(lambda x: clean_text(x))

In [77]:
# Токенизация
def tokenize_text(text):
    if isinstance(text, str):
      tokens = word_tokenize(text)
    return text

In [80]:
data['overview'] = data['overview'].apply(lambda x: tokenize_text(x))

In [78]:
# Лемматизация
def lemmatize_text(text):
    if isinstance(text, str):
      lemmatizer = WordNetLemmatizer()
      text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
    return text

In [79]:
data['overview'] = data['overview'].apply(lemmatize_text)

In [82]:
# Удаление стоп-слов
def remove_stopwords(text):
    if isinstance(text, str):
        stop_words = set(stopwords.words('english'))
        text = ' '.join([word for word in text.split() if word.lower() not in stop_words])
    return text

In [83]:
data['overview'] = data['overview'].apply(remove_stopwords)

In [85]:
data.head()

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,original_language,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,en,Inception,Cobb skilled thief commits corporate espionage...,83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili"
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,en,Interstellar,adventure group explorer make use newly discov...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,en,The Dark Knight,Batman raise stake war crime help Lt Jim Gordo...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin"
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,en,Avatar,nd century paraplegic Marine dispatched moon P...,79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish"
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,en,The Avengers,unexpected enemy emerges threatens global safe...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian"


In [86]:
data = data.dropna(subset=['overview'])

In [87]:
# Векторизация текста
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X = tfidf_vectorizer.fit_transform(data['overview'])
y = data['revenue']

In [88]:
selector = SelectKBest(score_func=f_regression, k=10)
X_new = selector.fit_transform(X, y)

# Получение индексов наиболее важных признаков
top_feature_indices = np.argsort(selector.scores_)[::-1][:10]
top_features = [tfidf_vectorizer.get_feature_names_out()[i] for i in top_feature_indices]
print("Наиболее важные слова, влияющие на доход фильма:", top_features)

Наиболее важные слова, влияющие на доход фильма: ['must', 'mission', 'earth', 'evil', 'world', 'battle', 'force', 'team', 'agent', 'film']


In [90]:
# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение модели (в данном случае, линейная регрессия)
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)

# Получение важности признаков (слов)
feature_names = tfidf_vectorizer.get_feature_names_out()
feature_importances = model.coef_

# Сортировка слов по важности
word_importance = sorted(zip(feature_names, feature_importances), key=lambda x: x[1], reverse=True)

# Вывод списка слов, влияющих на продвижение фильма
top_words = [word for word, importance in word_importance[:10]]
print("Слова, влияющие на продвижение фильма:", top_words)

Слова, влияющие на продвижение фильма: ['peter', 'protect', 'mission', 'epic', 'bond', 'meanwhile', 'jack', 'earth', 'quest', 'dangerous']


In [94]:
# Предсказание на тестовом наборе
y_pred = model.predict(X_test)

# Оценка точности
mse = mean_squared_error(y_test, y_pred)
print("Среднеквадратичная ошибка (MSE):", mse)

from sklearn.metrics import mean_squared_error, r2_score

# Коэффициент детерминации
r2 = r2_score(y_test, y_pred)
print("Коэффициент детерминации (R-squared):", r2)

Среднеквадратичная ошибка (MSE): 766052458825675.1
Коэффициент детерминации (R-squared): 0.011085565972789535


Использованный код анализирует влияние слов на доход фильма с использованием TF-IDF и линейной регрессии. Затем выводит список слов, которые имеют наибольшее влияние на доход фильма согласно построенной модели.

Наиболее важные слова, влияющие на доход фильма: ['must', 'mission', 'earth', 'evil', 'world', 'battle', 'force', 'team', 'agent', 'film']

Слова, влияющие на продвижение фильма: ['peter', 'protect', 'mission', 'epic', 'bond', 'meanwhile', 'jack', 'earth', 'quest', 'dangerous']